# get_stoichiometric_coefficients.ipynb

参与指数(Participation Index: PI)を取得するための前処理として，`src/c/spec_rates.c`のテキストを解析して，量論係数(stoich_coeffs_cema)，逆反応のインデックス(list_i_rev_rates)，圧力依存性のインデックス(list_k_pres_mod)を取得する．

In [1]:
import os
import numpy as np

## Get numbers of species/reactions from mechanism.h

In [2]:
path_h = '../src/c/mechanism.h'
print(os.path.exists(path_h))

True


In [3]:
with open(path_h, 'r', encoding='utf-8') as file:
    for line in file:
        if line.startswith('#define NSP'):
            n_species = int(line.split()[-1])
        
        if line.startswith('#define FWD_RATES'):
            n_reactions = int(line.split()[-1])

print('n_species:', n_species)
print('n_reactinos:', n_reactions)

n_species: 26
n_reactinos: 171


## Get indices from spec_rates.c

In [4]:
path_c = '../src/c/spec_rates.c'
print(os.path.exists(path_c))

True


In [5]:
stoich_coeffs_cema = np.zeros(shape=(n_species, n_reactions), dtype=np.int32)
list_k_pres_mod = np.zeros(shape=n_reactions, dtype=np.int32)
list_i_rev_rates = np.zeros(shape=n_reactions, dtype=np.int32)

with open(path_c, 'r', encoding='utf-8') as file:
        lines = file.readlines()

for line in lines:

      # pick reaction number
      if '//rxn' in line:
            i_reaction = int(line.split(' ')[-1])
            # print('reaction ', i_reaction)

      # pick species number
      if '//sp' in line:
            j_species = int(line.split(' ')[-1]) -1
            # print('species ', j_species)

      # pick coefficient
      coeff = 1.0
      
      if 'sp_rates' in line and '=' in line:

            if 'sp_rates[0]' in line:
                  continue
    
            sign_equals = line.split('=')[0][-1:]
            sign_bracket = line.split('(')[0][-1:]

            if sign_equals == '-':
                  coeff *= - 1.0
                  
            if sign_bracket == '-':
                  coeff *= - 1.0

            if '*' in line.split('=')[1].split('fwd_rates')[0]:
                  # print(float(line.split('=')[1].split('fwd_rates')[0].split('*')[0]))
                  coeff *= float(line.split('=')[1].split('fwd_rates')[0].split('*')[0])

            # print(coeff, line)
            stoich_coeffs_cema[j_species, i_reaction] = coeff

            if 'rev_rates' in line:
                  i_rev_rates = int(line.split('rev_rates[')[1].split(']')[0])
                  # print(i_rev_rates, line)
                  list_i_rev_rates[i_reaction] = i_rev_rates
            else:
                  list_i_rev_rates[i_reaction] = -1
                  
            
      
            if 'pres_mod' in line:
                  k_pres_mod = int(line.split('* pres_mod')[-1].split('[')[1].split(']')[0])
                  # print(k_pres_mod, i_reaction, line)
                  list_k_pres_mod[i_reaction] = k_pres_mod
            else:
                  list_k_pres_mod[i_reaction] = -1


## Check indices

In [6]:
stoich_coeffs_cema[:, -1]

array([ 0,  0,  0, -1,  0,  0,  0,  0, -1,  0,  0,  0,  0,  1,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0], dtype=int32)

In [7]:
list_i_rev_rates

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139,  -1, 140, 141,
       142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154,
       155, 156, 157, 158, 159, 160, 161, 162,  -1,  -1, 163, 164, 165,
       166, 167], dtype=int32)

In [8]:
list_k_pres_mod

array([-1, -1, -1, -1, -1, -1,  0,  1,  2,  3, -1,  4, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1,  5, -1, -1, -1, -1, -1,  6, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1,  7, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  8,  9, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, 10, -1, -1, -1, -1, -1, -1, -1, 11,
       12, -1, -1, -1, -1, -1, 13, -1, -1, 14, -1, -1, -1, -1, -1, -1, 15,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 16, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 17,
       18], dtype=int32)

## Export indices

In [9]:
stoich_coeffs_cema.reshape([n_species*n_reactions],order='F').tofile("stoich_coeffs.bin")
list_i_rev_rates.tofile("list_i_rev_rates.bin")
list_k_pres_mod.tofile("list_k_pres_mod.bin")